In [1]:
import numpy as np
import matplotlib.pyplot as plt
import utilis as u
import importlib
import model as m 

Using TensorFlow backend.


In [2]:
importlib.reload(u)
filename = '/data_batch_1'
X_train, y_train,Y_train = u.load_data(filename, reshape=False, clipping=True)
meanX = np.mean(X_train,axis=1)
stdX = np.std(X_train,axis=1)
X_train = (X_train-meanX.reshape((len(meanX),1)))/stdX.reshape((len(stdX),1))

filename = '/data_batch_2'
X_val, y_val,Y_val = u.load_data(filename, reshape=False, clipping=True)
X_val = (X_val-meanX.reshape((len(meanX),1)))/stdX.reshape((len(stdX),1))

filename = '/test_batch'
X_test, y_test,Y_test = u.load_data(filename, reshape=False, clipping=True)
X_test = (X_test-meanX.reshape((len(meanX),1)))/stdX.reshape((len(stdX),1))

data = {'X_train':X_train, 'Y_train':Y_train, 'y_train':y_train,'X_val':X_val, 'Y_val':Y_val, 'y_val':y_val, 'X_test':X_test, 'Y_test':Y_test, 'y_test':y_test}

In [4]:
importlib.reload(m)
mlp = m.MLP(lambda_=1e-3, dimensions=[100,60,30,10])
P = mlp.forward_batchnorm(X_train[:100,:1])
mlp.compute_gradients_batchnorm(X_train[:100,:1], Y_train[:,:1],P)
mlp.compare_gradients(X_train[:100,:1], Y_train[:,:1])

In [3]:
importlib.reload(m)
X_train_whole, y_train_whole, Y_train_whole = u.load_data('/data_batch_1', clipping=True)
for i in range(2,6):
    X, y, Y = u.load_data('/data_batch_'+str(i), clipping=True)
    X_train_whole = np.concatenate((X, X_train_whole), axis=1)
    y_train_whole = np.concatenate((y, y_train_whole))
    Y_train_whole = np.concatenate((Y, Y_train_whole), axis=1)

X_val_small, y_val_small, Y_val_small = X_train_whole[:,-5000:], y_train_whole[-5000:], Y_train_whole[:,-5000:]
X_train_whole, y_train_whole, Y_train_whole = X_train_whole[:,:-5000], y_train_whole[:-5000], Y_train_whole[:,:-5000]

filename = '/test_batch'
X_test_45, y_test,Y_test = u.load_data(filename, reshape=False, clipping=True)

## normalize with mean and std of train set 
mean = np.mean(X_train_whole, axis=1)
std = np.std(X_train_whole, axis=1)

X_train_whole -= np.outer(mean, np.ones(X_train_whole.shape[1]))
X_train_whole /= np.outer(std, np.ones(X_train_whole.shape[1]))

X_val_small -= np.outer(mean, np.ones(X_val_small.shape[1]))
X_val_small /= np.outer(std, np.ones(X_val_small.shape[1]))

X_test_45 -= np.outer(mean, np.ones(X_test_45.shape[1]))
X_test_45 /= np.outer(std, np.ones(X_test_45.shape[1]))


data_45 = {'X_train':X_train_whole, 'Y_train':Y_train_whole, 'y_train':y_train_whole,'X_val':X_val_small, 'Y_val':Y_val_small, 'y_val':y_val_small, 'X_test':X_test_45, 'Y_test':Y_test,'y_test':y_test}

In [7]:
importlib.reload(m)
ns = 5*45000/100
GD_params = {"n_batch":100, "eta_min":1e-5, 'eta_max':1e-1, 'ns':ns, 'n_cycles':2, 'freq':10}
dims = [3072,50,50,10]
for seed in range(3):
    mlp = m.MLP(lambda_=5e-3, dimensions=dims,seed=seed,layer_init='He')
    model = mlp.cyclicLearning(data_45, GD_params,True, '3_layer_with_batchnorm', False, True)
    mlp = m.MLP(lambda_=5e-3, dimensions=dims,seed=seed,layer_init='He')
    model = mlp.cyclicLearning(data_45, GD_params,False, '3_layer_without_batchnorm', False, True)

SyntaxError: invalid syntax (<ipython-input-7-a13e4c04d71c>, line 8)

In [ ]:
importlib.reload(m)
ns = 5*45000/100
GD_params = {"n_batch":100, "eta_min":1e-5, 'eta_max':1e-1, 'ns':ns, 'n_cycles':2, 'freq':10}
dims = [3072,50,30,20,20,10,10,10,10]
for seed in range(3):
    mlp = m.MLP(lambda_=5e-3, dimensions=dims,seed=seed,layer_init='He')
    model = mlp.cyclicLearning(data_45, GD_params,True, '9_layer_with_batchnorm', False, True)
    mlp = m.MLP(lambda_=5e-3, dimensions=dims,seed=seed,layer_init='He')
    model = mlp.cyclicLearning(data_45, GD_params,False, '9_layer_without_batchnorm', False, True)

In [9]:
importlib.reload(m)
ns = 2*10000/100
GD_params = {"n_batch":100, "eta_min":1e-5, 'eta_max':1e-1, 'ns':ns, 'n_cycles':1, 'freq':10}
dims = [3072,50,50,10]
for init in [1e-1,1e-3,1e-4]:
    mlp = m.MLP(lambda_=5e-3, dimensions=dims,layer_init=init)
    model = mlp.cyclicLearning(data, GD_params,True, f'init_{init}_with_batchnorm', False, True)
    mlp = m.MLP(lambda_=5e-3, dimensions=dims,layer_init=init)
    model = mlp.cyclicLearning(data, GD_params,False, f'init_{init}_without_batchnorm', False, True)

 39%|███▉      | 7/18 [00:33<00:50,  4.55s/it]

KeyboardInterrupt: 